In [11]:
import json
import torch
import torch.nn as nn

In [12]:
with open('filtered_data_cleaned.json', 'r') as j:
    posts = json.load(j)

post_data = {f'post{num+1}': {'post': post['content'], 'comment': [comment['content'] for comment in post['comments']]} for num, post in enumerate(posts)}

post_data

{'post1': {'post': '저희딸이3살인데 요즘 계속 기침을 하는데뭘먹이면 좋을까요?',
  'comment': ['면역력때문에 포베라아연먹이구있네요~',
   '애들 면역력 떨어지면 그렇더라구요.. 저희 애도 면역력 약한 편이라서 감기 잘 걸리는데 환절기되면 심하더라구요 기침도 많이 하구요ㅜㅜ 기관지 걱정도 돼서 전 배도라지즙 먹이고 있어요 지금 4색보감 배도라지즙 먹이고 있는데 달달해서 그런지 잘 먹고 있어요 배도라지즙 먹이니까 좀 낫더라구요',
   '저희 아이도 4색보감 배도라지즙 잘 먹더라구요',
   '기침자주하는 아이들은 평소에 관리해주시는게 정말 중요해용!!! 요즘보니 푸로, 한약 등등 이렇게 다양하게 관리해주시던데 저는 배도라지로 관래해주는게 젤 믿음직스럽더라구요 ㅎㅎㅎ 4색보감 배도라지즙 먹이고있는데 국내산 성분에 전문가가 만들어서 믿고 먹였어요 ㅎㅎㅎ 애들먹이는데 성분 중요하잖아요~ 국내산 생강, 배도라지 뿐만아니라 빌베리추출물이라고 눈건강에 좋은 성분도 있어서 1석2조였어요 ㅎㅎ',
   '4색보감 배도라지즙 괜찮나보네요 ㅎㅎㅎ',
   '저희집은 남편이 어릴때부터 비염, 감기 달고살았다더니 아들도 똑같아요 ㅠㅠㅠㅠ 그래서 아들만은 남편처럼되면 안되겠다 싶어서 이것저것 찾아보니 배도라지가 기관지 질병들에 좋다길래 찾아보는데 제품이 엄청 많더라구요~ 근데 4색보감 배도라지즙이 아이들 먹기좋대서 먹여봤는데 맛도 쓰지않고 달달해서 아이가 잘먹고 빌베리 추출물이라고 눈에좋은 성분도 들어있더라구요~ 이거먹고난뒤로 증상도 훨씬 많이 나아져서 예전에 비해 기관지 질병 하는 횟수가 줄었네요 ㅎㅎ',
   '배도라지청이용 ! ㅋㅋ']},
 'post2': {'post': '아들인데 아직 출산 예정인데요 혹시 신생아때 포경 수술 시키는거 있나요 ?? 커서 시키나요 ??',
  'comment': ['자연포경이 될 경우도 있기에 2차성징 끝나고, 비뇨기과 상담받은 후 결정하는게 좋다고해용!!!!(비뇨기과 의사왈)']},
 'post3': {'post

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer

MODEL_NAME = "BM-K/KoSimCSE-roberta" # "bert-base-multilingual-cased"

model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
sample_post_data = post_data.copy()

cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

for post_num in post_data:
    encoded_post = tokenizer(post_data[post_num]['post'], return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        post_outputs = model(**encoded_post)
        post_pooler_output = post_outputs.pooler_output
    for num, comment in enumerate(post_data[post_num]['comment']):
        encoded_comment = tokenizer(comment, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            comments_output = model(**encoded_comment)
            comments_pooler_output = comments_output.pooler_output
            similarity = cos_sim(post_pooler_output, comments_pooler_output).item()*100
        print(f"의미가 유사한 문장 간 유사도 계산: ({post_num}, comment{num+1}) = {similarity:.2f}%")
        if similarity < 60:
            sample_post_data[post_num]['comment'].remove(comment)

의미가 유사한 문장 간 유사도 계산: (post1, comment1) = 35.88%
의미가 유사한 문장 간 유사도 계산: (post1, comment2) = 57.03%
의미가 유사한 문장 간 유사도 계산: (post1, comment3) = 29.25%
의미가 유사한 문장 간 유사도 계산: (post1, comment4) = 46.88%
의미가 유사한 문장 간 유사도 계산: (post1, comment5) = 25.58%
의미가 유사한 문장 간 유사도 계산: (post1, comment6) = 40.54%
의미가 유사한 문장 간 유사도 계산: (post1, comment7) = 20.38%
의미가 유사한 문장 간 유사도 계산: (post2, comment1) = 32.28%
의미가 유사한 문장 간 유사도 계산: (post3, comment1) = 61.56%
의미가 유사한 문장 간 유사도 계산: (post4, comment1) = 23.82%
의미가 유사한 문장 간 유사도 계산: (post4, comment2) = 45.31%
의미가 유사한 문장 간 유사도 계산: (post4, comment3) = 53.74%
의미가 유사한 문장 간 유사도 계산: (post4, comment4) = 35.56%
의미가 유사한 문장 간 유사도 계산: (post4, comment5) = 14.25%
의미가 유사한 문장 간 유사도 계산: (post4, comment6) = 36.45%
의미가 유사한 문장 간 유사도 계산: (post4, comment7) = 34.04%
의미가 유사한 문장 간 유사도 계산: (post4, comment8) = 36.56%
의미가 유사한 문장 간 유사도 계산: (post5, comment1) = 44.25%
의미가 유사한 문장 간 유사도 계산: (post5, comment2) = 63.80%
의미가 유사한 문장 간 유사도 계산: (post6, comment1) = 31.31%
의미가 유사한 문장 간 유사도 계산: (post6, comment2) =

: 

In [ ]:
sample_post_data